In [39]:
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import json



In [40]:
dataset = pd.read_pickle("./prepared_ad.pkl")


In [41]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('PRICE')
test_labels = test_features.pop('PRICE')

In [42]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))


In [43]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.00005))
  return model

In [44]:
dnn_model = build_and_compile_model(normalizer)

In [45]:
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    epochs=2500)

Epoch 1/2500
26/26 [==============================] - 0s 5ms/step - loss: 52.7198 - val_loss: 52.6714
Epoch 2/2500
26/26 [==============================] - 0s 2ms/step - loss: 52.5306 - val_loss: 52.4840
Epoch 3/2500
26/26 [==============================] - 0s 2ms/step - loss: 52.3406 - val_loss: 52.2922
Epoch 4/2500
26/26 [==============================] - 0s 2ms/step - loss: 52.1488 - val_loss: 52.0989
Epoch 5/2500
26/26 [==============================] - 0s 2ms/step - loss: 51.9523 - val_loss: 51.9007
Epoch 6/2500
26/26 [==============================] - 0s 2ms/step - loss: 51.7514 - val_loss: 51.7010
Epoch 7/2500
26/26 [==============================] - 0s 2ms/step - loss: 51.5473 - val_loss: 51.4946
Epoch 8/2500
26/26 [==============================] - 0s 2ms/step - loss: 51.3338 - val_loss: 51.2817
Epoch 9/2500
26/26 [==============================] - 0s 2ms/step - loss: 51.1132 - val_loss: 51.0602
Epoch 10/2500
26/26 [==============================] - 0s 1ms/step - loss: 50.8845

In [48]:
dnn_model.save('price_predictor_model')


INFO:tensorflow:Assets written to: price_predictor_model/assets


In [47]:
with open('feature_order.json', 'w', encoding='utf-8') as f:
    json.dump(dataset.columns.values.tolist(), f, ensure_ascii=False, indent=4)